In [1]:
! ls /SSD/slava/brain_decoding/nsd/data/nsddata/ppdata/subj01/func1pt8mm/roi/

corticalsulc.nii.gz	lh.Kastner2015.nii.gz	  rh.floc-faces.nii.gz
floc-bodies.nii.gz	lh.MTL.nii.gz		  rh.floc-places.nii.gz
floc-faces.nii.gz	lh.nsdgeneral.nii.gz	  rh.floc-words.nii.gz
floc-places.nii.gz	lh.prf-eccrois.nii.gz	  rh.HCP_MMP1.nii.gz
floc-words.nii.gz	lh.prf-visualrois.nii.gz  rh.Kastner2015.nii.gz
HCP_MMP1.nii.gz		lh.streams.nii.gz	  rh.MTL.nii.gz
Kastner2015.nii.gz	lh.thalamus.nii.gz	  rh.nsdgeneral.nii.gz
lh.corticalsulc.nii.gz	MTL.nii.gz		  rh.prf-eccrois.nii.gz
lh.floc-bodies.nii.gz	nsdgeneral.nii.gz	  rh.prf-visualrois.nii.gz
lh.floc-faces.nii.gz	prf-eccrois.nii.gz	  rh.streams.nii.gz
lh.floc-places.nii.gz	prf-visualrois.nii.gz	  rh.thalamus.nii.gz
lh.floc-words.nii.gz	rh.corticalsulc.nii.gz	  streams.nii.gz
lh.HCP_MMP1.nii.gz	rh.floc-bodies.nii.gz	  thalamus.nii.gz


In [2]:
import os
import sys
from distutils.dir_util import copy_tree

import numpy as np
import h5py
import scipy.io as spio
import nibabel as nib

from glob import glob
import pandas as pd
import copy
from tqdm import tqdm

import pickle 

def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
    
def save_pickle(data, path):
    with open(path, 'wb') as file:
        pickle.dump(data, file)
    
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [ ]:
for subj in [1, 2, 5, 7]:
    train_fmri = load_pickle(f'/SSD/slava/THESIS/NSD_processed/subj{subj}_train_mri_to_nsd-img.pkl')
    test_fmri = load_pickle(f'/SSD/slava/THESIS/NSD_processed/subj{subj}_test_mri_to_nsd-img.pkl')

    betas_dir = '/SSD/slava/brain_decoding/nsd/data/nsddata_betas/ppdata/subj{:02d}/func1pt8mm/betas_fithrf_GLMdenoise_RR/'.format(subj)
    roi_dir = '/SSD/slava/brain_decoding/nsd/data/nsddata/ppdata/subj{:02d}/func1pt8mm/roi/'.format(subj)
    
    # depending on the subj
    if subj==1:
        shape = (1, 81, 104, 83)
    elif subj==2:
        shape = (1, 82, 106, 84)
    elif subj==5:
        shape = (1, 79, 97, 78)
    elif subj==7:
        shape = (1, 78, 95, 81)
    else:
        raise ValueError('Wrong subject')

    mean_arr = np.zeros(shape) 

    for i in tqdm(range(37), desc="Calculating mean"):
        beta_filename = "betas_session{0:02d}.nii.gz".format(i+1)
        beta_f = nib.load(betas_dir+beta_filename).get_fdata().astype(np.float32)

        beta_f_transposed = np.transpose(beta_f, axes=(3, 0, 1, 2))
        for j in range(beta_f_transposed.shape[0]):
            fmriId = (i*750) + j
            if fmriId in train_fmri:
                mean_arr += beta_f_transposed[j, :, :, :]/300

        del beta_f, beta_f_transposed

    mean_arr = mean_arr/len(train_fmri)
    print("Mean is calculated for Subject ", subj)
    
    print("Mean unique values: ", np.unique(mean_arr))
    
    std_arr = np.zeros(shape)
    ddof = 1

    for i in tqdm(range(37), desc="Calculating std"):
        beta_filename = "betas_session{0:02d}.nii.gz".format(i+1)
        beta_f = nib.load(betas_dir+beta_filename).get_fdata().astype(np.float32)

        beta_f_transposed = np.transpose(beta_f, axes=(3, 0, 1, 2))

        for j in range(beta_f_transposed.shape[0]):
            fmriId = (i*750) + j
            if fmriId in train_fmri:
                std_arr += np.abs((beta_f_transposed[j, :, :, :]/300) - mean_arr) ** 2

        del beta_f, beta_f_transposed

    std_arr = np.sqrt(std_arr/(len(train_fmri) - ddof))
    
    print("Std is calculated for Subject ", subj)
    print("Std unique values: ", np.unique(std_arr))
    
    print('Saving mean and std')
    # save these stats
    np.save("/SSD/slava/THESIS/NSD_processed/subj{:02d}/train_mean.npy".format(subj),
           mean_arr)

    np.save("/SSD/slava/THESIS/NSD_processed/subj{:02d}/train_std.npy".format(subj),
           std_arr)
    
    
    save_dir = "/SSD/slava/THESIS/NSD_processed/subj{:02d}/fMRI_frames_normalized".format(subj)

    for i in tqdm(range(37), desc="Saving fMRI"):
        beta_filename = "betas_session{0:02d}.nii.gz".format(i+1)
        beta_f = nib.load(betas_dir+beta_filename).get_fdata().astype(np.float32)

        beta_f_transposed = np.transpose(beta_f, axes=(3, 0, 1, 2))

        for j in range(beta_f_transposed.shape[0]):
            fmriId = (i*750) + j
            normalized = (beta_f_transposed[j, :, :, :]/300 - mean_arr) / (std_arr + 1e-6)

            if fmriId in train_fmri:
                split = 'train'
            elif fmriId in test_fmri:
                split = 'test'

            np.save(
                   os.path.join(save_dir, f'fmri_{fmriId}_{split}.npy'),
                   normalized
            )
            
    
    print('Copying ROIs')
    # copy rois
    copy_tree(roi_dir,
             "/SSD/slava/THESIS/NSD_processed/subj{:02d}/roi".format(subj))
    
    print("Done for subj ", subj)
    


Calculating mean:   5%|████▎                                                                          | 2/37 [00:21<06:14, 10.71s/it]

In [5]:
# check number with assert
for subj in [1, 2, 5, 7]:
    train_fmri = load_pickle(f'/SSD/slava/THESIS/NSD_processed/subj{subj}_train_mri_to_nsd-img.pkl')
    test_fmri = load_pickle(f'/SSD/slava/THESIS/NSD_processed/subj{subj}_test_mri_to_nsd-img.pkl')

    train_files = glob(f'/SSD/slava/THESIS/NSD_processed/subj0{subj}/fMRI_frames_normalized/*_train.npy')
    test_files = glob(f'/SSD/slava/THESIS/NSD_processed/subj0{subj}/fMRI_frames_normalized/*_test.npy')
    
    assert len(train_files)==len(train_fmri)
    assert len(test_files)==len(test_fmri)